#### Import libraries

In [41]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

#### Set up the connection for SQL Server

In [98]:
server = r'localhost\SQLEXPRESS' 
database = 'audiobook'
username = 'sa'
password = '123'
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

# Create the engine
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


#### Load the csv file into a dataframe

In [ ]:
df = pd.read_csv(r'C:\Users\bruno\OneDrive\01. Data\03. Python\03. Audiobooks\audible_uncleaned.csv')

## Data  quality check

#### Completeness

##### Check if there are any missing data

In [51]:
df.isnull().sum()

name           0
author         0
narrator       0
time           0
releasedate    0
language       0
stars          0
price          0
dtype: int64

#### Consistency
##### Check if data formatting is consistent across columns.

In [54]:
author_prefix_check = df['author'].str.startswith("Writtenby:")
print(df[~author_prefix_check])

Empty DataFrame
Columns: [name, author, narrator, time, releasedate, language, stars, price]
Index: []


In [56]:
narrator_prefix_check = df['narrator'].str.startswith("Narratedby:")
print(df[~narrator_prefix_check])

Empty DataFrame
Columns: [name, author, narrator, time, releasedate, language, stars, price]
Index: []


##### Remove the Writtenby and Narratedby from author and narrator column

In [59]:
df['author'] = df['author'].str.replace('Writtenby:','')
df['narrator'] = df['narrator'].str.replace('Narratedby:','')
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,JeffKinney,DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [61]:
time_split = df['time'].str.split(' ', expand = True)
time_split.head()

,0,1,2,3,4
0,2,hrs,and,20,mins
1,13,hrs,and,8,mins
2,2,hrs,and,3,mins
3,11,hrs,and,16,mins
4,10,hrs,None,None,None


In [62]:
df['hour'] = time_split[0]
df['minute'] = time_split[3]
df.head()

,name,author,narrator,time,releasedate,language,stars,price,hour,minute
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00,2,20
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00,13,8
2,The Deep End,JeffKinney,DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00,2,3
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00,11,16
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00,10,None


##### Convert hour and time into numeric 

In [66]:
df['hour'] = pd.to_numeric(df['hour'], errors = 'coerce')
df['minute'] = pd.to_numeric(df['minute'], errors = 'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         87489 non-null  object 
 1   author       87489 non-null  object 
 2   narrator     87489 non-null  object 
 3   time         87489 non-null  object 
 4   releasedate  87489 non-null  object 
 5   language     87489 non-null  object 
 6   stars        87489 non-null  object 
 7   price        87489 non-null  object 
 8   hour         87428 non-null  float64
 9   minute       72740 non-null  float64
dtypes: float64(2), object(8)
memory usage: 6.7+ MB


##### Replace NaN values

In [69]:
df['hour'] = df['hour'].fillna(0)
df['minute'] = df['minute'].fillna(0)
df.head()

,name,author,narrator,time,releasedate,language,stars,price,hour,minute
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00,2.0,20.0
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00,13.0,8.0
2,The Deep End,JeffKinney,DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00,2.0,3.0
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00,11.0,16.0
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00,10.0,0.0


In [71]:
df['time'] = df['hour'].astype(int).astype(str).str.zfill(2) + ':' + df['minute'].astype(int).astype(str).str.zfill(2)
df = df.drop(columns=['hour','minute'])
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,02:20,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,RickRiordan,RobbieDaymond,13:08,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,JeffKinney,DanRussell,02:03,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11:16,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10:00,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


##### Drop hour and minute columns

##### Release date

In [75]:
df['releasedate'] = pd.to_datetime(df['releasedate'], format='%d-%m-%y', errors='coerce')
df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,02:20,2008-08-04,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,RickRiordan,RobbieDaymond,13:08,2018-05-01,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,JeffKinney,DanRussell,02:03,2020-11-06,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11:16,2021-10-05,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10:00,2010-01-13,English,4.5 out of 5 stars181 ratings,820.00


##### Split rating and stars

In [78]:
stars_split = df['stars'].str.split('stars', expand = True)
stars_split.head()

,0,1
0,5 out of 5,34 ratings
1,4.5 out of 5,41 ratings
2,4.5 out of 5,38 ratings
3,4.5 out of 5,12 ratings
4,4.5 out of 5,181 ratings


##### Create stars column

In [81]:
#Split the column by each space occurrence.  
stars = stars_split[0].str.split(' ', expand = True) 

#Extract the first index, which represents the number of stars received. 
df['stars'] = stars[0]

#Convert the extracted value to a number
df['stars'] = pd.to_numeric(df['stars'],errors = 'coerce')

#Replace any entries where the audiobook has not been rated with 0
df['stars'] = df['stars'].fillna(0)

df.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,02:20,2008-08-04,English,5.0,468.00
1,The Burning Maze,RickRiordan,RobbieDaymond,13:08,2018-05-01,English,4.5,820.00
2,The Deep End,JeffKinney,DanRussell,02:03,2020-11-06,English,4.5,410.00
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11:16,2021-10-05,English,4.5,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10:00,2010-01-13,English,4.5,820.00


##### Create ratings column

In [84]:
#Remove the word "ratings."
df['ratings'] = stars_split[1].str.replace(' ratings','')

#Convert the remaining value to a number.
df['ratings'] = pd.to_numeric(df['ratings'], errors = 'coerce')

#Replace any entries where the audiobook has not been rated with 0.
df['ratings'] = df['ratings'].fillna(0)

df.head()

,name,author,narrator,time,releasedate,language,stars,price,ratings
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,02:20,2008-08-04,English,5.0,468.00,34.0
1,The Burning Maze,RickRiordan,RobbieDaymond,13:08,2018-05-01,English,4.5,820.00,41.0
2,The Deep End,JeffKinney,DanRussell,02:03,2020-11-06,English,4.5,410.00,38.0
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11:16,2021-10-05,English,4.5,615.00,12.0
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10:00,2010-01-13,English,4.5,820.00,181.0


In [104]:
# Load DataFrame into SQL Server table
df.to_sql('audiobook', con=engine, if_exists='replace', index=False)

114